In [ ]:
from collections import Counter
import numpy as np

In [ ]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

In [ ]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

In [ ]:
def _mcSquillero(board, player):
#funzione che esegue il resto della partita randomicamente, il risultato della simulazione aiuta a selezionare la mossa successiva

    p = -player
    while valid_moves(board):
    #ciclo per completare la partita
    
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
        #controlla se vince qualcuno
            return p
    return 0

def montecarloSquillero(board, player, plays):
#funzione per lanciare un campione di partite random (uguale a quella postata su git dal professore)
    montecarlo_samples = plays
    cnt = Counter(_mcSquillero(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples

def tryMove(board, player):
    #funzione per testare la fitness della mossa, si basa sulle funzioni riportate in alto per ottenere un valore da assegnare alla mossa
    mov = montecarloSquillero(board, player, 25)*player
    #la moltiplicazione per player serve a normalizzare i valori su una scala da -1 a +1 dove -1 è la fitness minima e +1 quella massima
    #valore generico molto basso da sovrascrivere
    counterBest = -20
    for c in valid_moves(board):
        #per non rifarla
        fakeBoard = np.copy(board)
        #fai la mossa di prova (risposta)
        play(fakeBoard, c, -player)
        tmp = montecarloSquillero(fakeBoard, -player, 20)*(-player)
        #la moltiplicazione per player serve a normalizzare i valori su una scala da -1 a +1 dove -1 è la fitness minima e +1 quella massima
        if tmp == 1.0:
            #se per una mossa la risposta dell'avversario porta sicuramente a una sconfitta, la fitness viene forzata a un valore molto alto
            #per sottolineare che quella mossa deve essere evitata a tutti i costi
            tmp == 10
        if counterBest < tmp :
            #peggiore fitness tra le simulazioni, considerare sempre worst case scenario
            counterBest = tmp
    #la differenza tra la fitness della mossa e quella della risposta migliore sarà la fitness effettiva della mossa        
    return (mov - counterBest)

def _mc(board, player):
#selezione della mossa e verifica delle fitness
    best = -20
    col = 0
    for c in valid_moves(board):
        #per non rifarla ogni ciclo
        fakeBoard = np.copy(board)
        #seleziona la mossa disponibile e gioca
        play(fakeBoard, c, player)
        #calcola fitness della mossa
        tmp = tryMove(fakeBoard, player)
        if best < tmp :
            best = tmp
            col = c
    #aggiorna la board reale
    play(board, col, player)
    if four_in_a_row(board, player):
        #controlla se vince qualcuno
        return player
    return 0    
    
def eval_board(board, player):
    if four_in_a_row(board, 1):
    # Alice won
        return 1
    elif four_in_a_row(board, -1):
    # Bob won
        return -1
    else:
    # Not terminal, let's simulate...
        return _mc(board, player)

In [ ]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)

print(board)
print('\n')
p=-1
tmp = 0
while tmp!=1 and tmp!=-1:
    #loop per giocare da solo :c
    p=-p
    tmp = eval_board(board, p)
    print(board)
    print('\n')
